In [1]:
import numpy as np
from multiple_factor import DGP2, Inferece2
from dgp import DGP

def reject_prob(Xdim, num_factor, sample_size, tau=0, ntrials=1000, more=False, design='MT'):
    phi_tau = np.zeros(ntrials)
    for i in range(ntrials):
        dgp = DGP2(num_factor, sample_size, Xdim, tau, more, design)
        Y, D, tuple_idx = dgp.Y, dgp.D, dgp.tuple_idx
        inf = Inferece2(Y, D, tuple_idx, design)
        phi_tau[i] = inf.phi_tau
    return np.mean(phi_tau)


def risk(Xdim, num_factor, sample_size, tau=0, ntrials=1000, more=False, design='MT'):
    mse = np.zeros(ntrials)
    for i in range(ntrials):
        dgp = DGP2(num_factor, sample_size, Xdim, tau, more, design)
        Y, D, tuple_idx = dgp.Y, dgp.D, dgp.tuple_idx
        ate = np.mean(Y[D[:,0]==1]) - np.mean(Y[D[:,0]==0])
        mse[i] = (ate - tau)**2
    return np.mean(mse)


def test(q,k):
    print(q,k)
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='S4'))
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='C'))
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='RE'))
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='MP-B'))
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='MT'))

#[test(q,k) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]]
#test(1,2)
#test(2,2)
#test(10,6)

In [2]:
print([(q,k,reject_prob(q,k,1280,tau=0,ntrials=1000,more=False, design='C')) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]])

[(1, 1, 0.061), (1, 2, 0.068), (1, 3, 0.071), (1, 4, 0.049), (1, 5, 0.063), (1, 6, 0.046), (2, 1, 0.047), (2, 2, 0.046), (2, 3, 0.051), (2, 4, 0.065), (2, 5, 0.056), (2, 6, 0.059), (4, 1, 0.055), (4, 2, 0.048), (4, 3, 0.041), (4, 4, 0.055), (4, 5, 0.064), (4, 6, 0.053), (8, 1, 0.048), (8, 2, 0.064), (8, 3, 0.052), (8, 4, 0.052), (8, 5, 0.057), (8, 6, 0.056), (10, 1, 0.045), (10, 2, 0.069), (10, 3, 0.047), (10, 4, 0.049), (10, 5, 0.05), (10, 6, 0.072)]


In [3]:
print([(q,k,reject_prob(q,k,1280,tau=0.02,ntrials=1000,more=False, design='C')) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]])

[(1, 1, 0.226), (1, 2, 0.225), (1, 3, 0.205), (1, 4, 0.201), (1, 5, 0.242), (1, 6, 0.243), (2, 1, 0.08), (2, 2, 0.089), (2, 3, 0.072), (2, 4, 0.09), (2, 5, 0.093), (2, 6, 0.086), (4, 1, 0.067), (4, 2, 0.075), (4, 3, 0.08), (4, 4, 0.08), (4, 5, 0.073), (4, 6, 0.076), (8, 1, 0.082), (8, 2, 0.071), (8, 3, 0.072), (8, 4, 0.06), (8, 5, 0.065), (8, 6, 0.065), (10, 1, 0.057), (10, 2, 0.053), (10, 3, 0.054), (10, 4, 0.057), (10, 5, 0.065), (10, 6, 0.066)]


In [4]:
print([(q,k,reject_prob(q,k,1280,tau=0,ntrials=1000,more=False, design='S4')) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]])

[(1, 1, 0.052), (1, 2, 0.071), (1, 3, 0.059), (1, 4, 0.057), (1, 5, 0.066), (1, 6, 0.083), (2, 1, 0.049), (2, 2, 0.045), (2, 3, 0.059), (2, 4, 0.056), (2, 5, 0.05), (2, 6, 0.083), (4, 1, 0.069), (4, 2, 0.046), (4, 3, 0.05), (4, 4, 0.058), (4, 5, 0.075), (4, 6, 0.068), (8, 1, 0.044), (8, 2, 0.058), (8, 3, 0.056), (8, 4, 0.062), (8, 5, 0.05), (8, 6, 0.077), (10, 1, 0.047), (10, 2, 0.048), (10, 3, 0.053), (10, 4, 0.063), (10, 5, 0.063), (10, 6, 0.059)]


In [5]:
print([(q,k,reject_prob(q,k,1280,tau=0.02,ntrials=1000,more=False, design='S4')) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]])

[(1, 1, 0.827), (1, 2, 0.824), (1, 3, 0.841), (1, 4, 0.836), (1, 5, 0.849), (1, 6, 0.873), (2, 1, 0.361), (2, 2, 0.378), (2, 3, 0.396), (2, 4, 0.389), (2, 5, 0.417), (2, 6, 0.439), (4, 1, 0.201), (4, 2, 0.2), (4, 3, 0.198), (4, 4, 0.198), (4, 5, 0.228), (4, 6, 0.292), (8, 1, 0.136), (8, 2, 0.132), (8, 3, 0.143), (8, 4, 0.125), (8, 5, 0.126), (8, 6, 0.179), (10, 1, 0.102), (10, 2, 0.099), (10, 3, 0.104), (10, 4, 0.103), (10, 5, 0.125), (10, 6, 0.15)]


designs = ['MT', 'C', 'MT2', 'S4', 'MP-B', 'RE']
results = []

for m in designs:
    print(m)
    if m == 'MT2':
        result = {(q,k): risk(q, k, 1280, tau=0, ntrials=1000, more=True, design='MT') 
         for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]}
    else:
        result = {(q,k): risk(q, k, 1280, tau=0, ntrials=1000, more=False, design=m) 
         for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]}
    results.append(result)
    baseline = results[0][(1,1)]
    for q in [1,2,4,8,10]:
        for k in [1,2,3,4,5,6]:
            if k<6:
                print("{:.3f} & ".format(result[(q,k)]/baseline), end = '')
            else:
                print("{:.3f} \\\\".format(result[(q,k)]/baseline))

In [9]:
from joblib import Parallel, delayed
import multiprocessing
# what are your inputs, and what operation do you want to
# perform on each input. For example...
designs = ['MT', 'C', 'MT2', 'S4', 'MP-B', 'RE']
results = []

num_cores = multiprocessing.cpu_count()
for m in designs:
    print(m)
    qk_pairs = [(q,k) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]]
    def processInput(qk):
        q, k = qk
        if m == 'MT2':
            mse = risk(q, k, 1280, tau=0, ntrials=1000, more=True, design='MT')
        else:
            mse = risk(q, k, 1280, tau=0, ntrials=1000, more=False, design=m) 
        return (q,k,mse)
    ret = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in qk_pairs)
    result = {(re[0], re[1]):re[2] for re in ret}
    results.append(result)
    baseline = results[0][(1,1)]
    for q in [1,2,4,8,10]:
        for k in [1,2,3,4,5,6]:
            if k<6:
                print("{:.3f} & ".format(result[(q,k)]/baseline), end = '')
            else:
                print("{:.3f} \\\\".format(result[(q,k)]/baseline))

MT


/Users/liujizhou/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


1.000 & 0.910 & 1.027 & 0.994 & 1.061 & 0.982 \\
1.083 & 1.082 & 1.206 & 1.598 & 2.078 & 3.530 \\
2.877 & 4.193 & 6.376 & 10.833 & 14.934 & 23.328 \\
18.529 & 27.173 & 43.575 & 55.882 & 72.740 & 99.039 \\
34.632 & 47.498 & 66.157 & 86.313 & 105.050 & 133.200 \\
C
8.719 & 9.915 & 8.939 & 8.688 & 9.160 & 9.372 \\
39.862 & 40.833 & 41.964 & 43.136 & 40.743 & 41.661 \\
82.635 & 80.093 & 77.194 & 76.094 & 77.294 & 75.385 \\
150.794 & 151.074 & 158.889 & 153.514 & 162.579 & 152.250 \\
191.086 & 198.797 & 198.302 & 187.129 & 186.745 & 189.368 \\
MT2
0.952 & 0.956 & 0.932 & 1.029 & 1.033 & 0.913 \\
1.077 & 1.300 & 1.490 & 2.279 & 3.393 & 6.092 \\
4.216 & 6.199 & 10.510 & 15.420 & 22.640 & 32.465 \\
29.525 & 42.196 & 54.750 & 73.479 & 93.407 & 107.663 \\
52.360 & 67.613 & 85.839 & 105.517 & 130.841 & 152.691 \\
S4
1.364 & 1.473 & 1.527 & 1.474 & 1.460 & 1.510 \\
4.725 & 4.784 & 5.155 & 4.574 & 4.365 & 4.690 \\
10.085 & 10.755 & 10.339 & 10.887 & 9.315 & 10.587 \\
20.741 & 20.319 & 22.911 & 21.1

MT
1.000 & 0.910 & 1.027 & 0.994 & 1.061 & 0.982 \\
1.083 & 1.082 & 1.206 & 1.598 & 2.078 & 3.530 \\
2.877 & 4.193 & 6.376 & 10.833 & 14.934 & 23.328 \\
18.529 & 27.173 & 43.575 & 55.882 & 72.740 & 99.039 \\
34.632 & 47.498 & 66.157 & 86.313 & 105.050 & 133.200 \\
C
8.719 & 9.915 & 8.939 & 8.688 & 9.160 & 9.372 \\
39.862 & 40.833 & 41.964 & 43.136 & 40.743 & 41.661 \\
82.635 & 80.093 & 77.194 & 76.094 & 77.294 & 75.385 \\
150.794 & 151.074 & 158.889 & 153.514 & 162.579 & 152.250 \\
191.086 & 198.797 & 198.302 & 187.129 & 186.745 & 189.368 \\
MT2
0.952 & 0.956 & 0.932 & 1.029 & 1.033 & 0.913 \\
1.077 & 1.300 & 1.490 & 2.279 & 3.393 & 6.092 \\
4.216 & 6.199 & 10.510 & 15.420 & 22.640 & 32.465 \\
29.525 & 42.196 & 54.750 & 73.479 & 93.407 & 107.663 \\
52.360 & 67.613 & 85.839 & 105.517 & 130.841 & 152.691 \\
S4
1.364 & 1.473 & 1.527 & 1.474 & 1.460 & 1.510 \\
4.725 & 4.784 & 5.155 & 4.574 & 4.365 & 4.690 \\
10.085 & 10.755 & 10.339 & 10.887 & 9.315 & 10.587 \\
20.741 & 20.319 & 22.911 & 21.179 & 21.350 & 22.597 \\
25.634 & 26.796 & 27.102 & 28.007 & 26.584 & 26.322 \\
MP-B
0.934 & 8.427 & 9.287 & 9.117 & 8.954 & 8.824 \\
1.051 & 42.443 & 39.211 & 41.895 & 40.500 & 41.139 \\
2.824 & 76.357 & 79.468 & 80.368 & 80.508 & 77.406 \\
19.699 & 155.502 & 146.029 & 162.590 & 156.694 & 156.089 \\
31.869 & 196.989 & 184.882 & 179.497 & 181.828 & 189.762 \\
RE
1.263 & 9.483 & 8.657 & 9.139 & 8.447 & 9.406 \\
4.468 & 45.670 & 38.905 & 41.420 & 39.329 & 43.900 \\
11.519 & 79.226 & 84.796 & 78.259 & 77.863 & 84.988 \\
36.042 & 153.676 & 152.059 & 144.937 & 154.131 & 143.505 \\
52.833 & 204.490 & 190.114 & 202.368 & 186.888 & 180.323 \\
